In [1]:
import scipy.io as sio
import tensorflow as tf
from tensorflow.keras import regularizers
import larq as lq
import numpy as np
from scipy.special import logsumexp

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print(np.shape(train_images))

(60000, 28, 28)


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255, test_images / 255

In [4]:
# sio.savemat('data.mat', {'train_images':train_images, 'train_labels':train_labels,
#                               'test_images':test_images, 'test_labels':test_labels})

In [36]:
thresholdVal = 0.05
layerSize = 500
kwargs0 = dict(kernel_regularizer=tf.keras.regularizers.l1(0.0001))
kwargs1 = dict(kernel_regularizer=tf.keras.regularizers.l1(0.0001))
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer=lq.quantizers.SteTern(threshold_value=thresholdVal,
                                ternary_weight_networks=False,
                                clip_value=1.0),
              kernel_constraint="weight_clip")
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    lq.layers.QuantDense(layerSize, kernel_quantizer=lq.quantizers.SteTern(threshold_value=thresholdVal,
                                ternary_weight_networks=False,
                                clip_value=1.0),
                                kernel_constraint="weight_clip",
                                use_bias=False, **kwargs0),
    tf.keras.layers.BatchNormalization(scale=False, center=False),
    lq.layers.QuantDense(layerSize, use_bias=False, **kwargs, **kwargs0),
    tf.keras.layers.BatchNormalization(scale=False, center=False),
    lq.layers.QuantDense(10, use_bias=False,input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip"),
    #tf.keras.layers.BatchNormalization(scale=False, center=False),
    tf.keras.layers.Activation("softmax")])

In [37]:
lq.models.summary(model3)

+sequential_2 stats------------------------------------------------------------------------------------------------------+
| Layer                  Input prec.    Outputs  # 1-bit  # 2-bit  # 32-bit  Memory  1-bit MACs  2-bit MACs  32-bit MACs |
|                              (bit)                 x 1      x 1       x 1    (kB)                                      |
+------------------------------------------------------------------------------------------------------------------------+
| flatten_2                        -  (-1, 784)        0        0         0       0           0           0            0 |
| quant_dense_6                    -  (-1, 500)        0   392000         0   95.70           0           0       392000 |
| batch_normalization_4            -  (-1, 500)        0        0      1000    3.91           0           0            0 |
| quant_dense_7                    1  (-1, 500)        0   250000         0   61.04           0      250000            0 |
| batch_normaliz

In [38]:
# tf.random.set_seed(2020)
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model3.fit(train_images, train_labels, batch_size=64, epochs=2)
test_loss, test_acc = model3.evaluate(test_images, test_labels, verbose=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 20s 327us/sample - loss: 4.9381 - accuracy: 0.8742
Epoch 2/2
60000/60000 [==============================] - 18s 304us/sample - loss: 3.2961 - accuracy: 0.9052
10000/1 - 2s - loss: 1.9563 - accuracy: 0.9062


In [39]:
def fgsm(model, input_image, input_label, epsilon):
    # Your implementation here
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        predictions = model(input_image)
        loss = loss_object(input_label, predictions)
    
    gradients = tape.gradient(loss, input_image)
    output_image = tf.clip_by_value(input_image + epsilon * tf.sign(gradients),
                                   clip_value_min=0, clip_value_max=1)
    return output_image

In [43]:
adversarial = True
model3.compile(optimizer='adadelta',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
if (adversarial):
    # adversarial training
    epochs = 2 
    for i in range(epochs):
        model3.fit(train_images, train_labels, batch_size=64, epochs=1)
        tensor_images = tf.convert_to_tensor(test_images)
        tensor_one_hot_labels = tf.convert_to_tensor(tf.one_hot(test_labels, 10))
        perturbed_images = fgsm(model3, tensor_images, tensor_one_hot_labels, 0.01)
        model3.fit(perturbed_images, test_labels, batch_size=64, epochs=1)

    test_loss, test_acc = model3.evaluate(test_images, test_labels, verbose=2)

Train on 60000 samples
60000/60000 [==============================] - 20s 337us/sample - loss: 1.9854 - accuracy: 0.9430
Train on 10000 samples
10000/10000 [==============================] - 4s 351us/sample - loss: 2.2809 - accuracy: 0.9304
Train on 60000 samples
60000/60000 [==============================] - 20s 331us/sample - loss: 1.9364 - accuracy: 0.9444
Train on 10000 samples
10000/10000 [==============================] - 3s 325us/sample - loss: 2.2071 - accuracy: 0.9304
10000/1 - 2s - loss: 1.2893 - accuracy: 0.9484


In [44]:
print(perturbed_images.shape)

(10000, 28, 28, 1)


In [46]:
def SteTernWeight(weight, threshold=0.05):
    (m, n) = np.shape(weight)
    for i in range(m):
        for j in range(n):
            if (weight[i, j] >= threshold):
                weight[i,j] = 1
            elif (weight[i, j] >= -threshold):
                weight[i, j] = 0
            else:
                weight[i, j] = -1
    return weight
                

def SignVec(vec):
    vec1 = (vec >= 0)
    vec1 = 2 * vec1 - 1
    return vec1

In [47]:
testData = test_images[1, :, :, :].reshape(28*28,1)
weight1 = model3.layers[1].get_weights()[0]
weight2 = model3.layers[3].get_weights()[0]
weight3 = model3.layers[5].get_weights()[0]

mean1 = model3.layers[2].get_weights()[0].reshape(layerSize, 1)
# offset1 = model3.layers[2].get_weights()[1].reshape(layerSize, 1)
variance1 = model3.layers[2].get_weights()[1].reshape(layerSize, 1)

mean2 = model3.layers[4].get_weights()[0].reshape(layerSize, 1)
# offset2 = model3.layers[4].get_weights()[1].reshape(layerSize, 1)
variance2 = model3.layers[4].get_weights()[1].reshape(layerSize, 1)

weight1 = SteTernWeight(weight1, threshold=thresholdVal)
weight2 = SteTernWeight(weight2, threshold=thresholdVal)
weight3 = SteTernWeight(weight3, threshold=0)

l1 = np.matrix(weight1).T * testData
l1 = (np.array(l1) - mean1) #/ np.sqrt(variance1)
#l1 = np.sqrt(variance1) * np.array(l1) + mean1
l1 = SignVec(l1)

l2 = np.matrix(weight2).T * l1
l2 = (np.array(l2) - mean2) #/ np.sqrt(variance2)
#l2 = np.sqrt(variance2) * np.array(l2) + mean2
l2 = SignVec(l2)

l3 = np.matrix(weight3).T * l2

print(l3)

print(np.exp(l3-logsumexp(l3)))

[[ 22.]
 [ 40.]
 [ 80.]
 [ 28.]
 [-62.]
 [ -2.]
 [ 18.]
 [-60.]
 [ 38.]
 [-88.]]
[[6.47023493e-26]
 [4.24835426e-18]
 [1.00000000e+00]
 [2.61027907e-23]
 [2.13886596e-62]
 [2.44260074e-36]
 [1.18506486e-27]
 [1.58042006e-61]
 [5.74952226e-19]
 [1.09276566e-73]]


In [26]:
testData = test_images[0, :, :, :].reshape(28*28, order="A")
print(testData[200:240])

[0.         0.         0.32941176 0.7254902  0.62352941 0.59215686
 0.23529412 0.14117647 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.87058824 0.99607843 0.99607843 0.99607843 0.99607843 0.94509804
 0.77647059 0.77647059 0.77647059 0.77647059]


In [48]:
np.sum(weight1 == 0)/ np.size(weight1)

0.9709336734693877

In [49]:
np.sum(weight2 == 0) / np.size(weight2)

0.929272

In [50]:
np.sum(weight3 == 0) / np.size(weight3)

0.0

In [51]:
model3.predict(test_images[1:2, :, :, :])

array([[6.4702353e-26, 4.2483541e-18, 1.0000000e+00, 2.6102794e-23,
        0.0000000e+00, 2.4426009e-36, 1.1850651e-27, 0.0000000e+00,
        5.7495231e-19, 0.0000000e+00]], dtype=float32)

In [52]:
mean3 = np.zeros(np.shape(weight3)[1])
upper = np.ones(np.shape(weight1)[0])
lower = np.zeros(np.shape(weight1)[0])
modelToSave = [{"name":"Flatten1", "type":"flatten", "inputSize": (28,28,1)},
               {"name":"QuantDense1", "type":"dense", "weights": weight1.T, "bias": -mean1,
                "activation":"Sign","upper":upper, "lower":lower}, 
               {"name":"QuantDense2", "type":"denseBin", "weights": weight2.T, "bias": -mean2, "activation":"Sign"},
               {"name":"Dense", "type":"denseBin", "weights": weight3.T, "bias": mean3}]
sio.savemat('nn2F500AllSparseAdv.mat', {'nn':modelToSave})
#print(modelToSave[1])

In [17]:
mean3

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])